# 02 - Preprocessing YouTube Toxic Comments

En este notebook realizamos el **preprocesamiento** de los comentarios de YouTube
para dejar un CSV limpio y listo para el modelado posterior.

Objetivos:

- Cargar el dataset original usado en `01_EDA.ipynb`.
- Definir la columna de texto y las columnas de etiquetas.
- Crear una etiqueta binaria agregada (`IsAnyToxic`).
- Limpiar el texto con expresiones regulares.
- Tokenizar, eliminar *stopwords* y lematizar usando spaCy.
- Generar una columna de texto final `text_processed`.
- Guardar un CSV preprocesado para ser usado en `03_Modeling.ipynb`,
  donde se generarán los PKL (vectorizador, modelo, etc.).


## 1. Imports y configuración

Importamos las librerías necesarias:

- `pandas`, `numpy`: manejo de datos.
- `re`: expresiones regulares para limpieza de texto.
- `spacy`: tokenización y lematización.
- `pathlib`: manejo de rutas.

> **Nota:** Antes de ejecutar este notebook puede ser necesario instalar
> el modelo de spaCy con:  
> `python -m spacy download en_core_web_sm`


In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
from pathlib import Path

import numpy as np
import pandas as pd

import spacy

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2. Carga de datos

Cargamos el dataset original que ya hemos explorado en el notebook de EDA.

> Ajusta `DATA_PATH` a la ruta real donde tengas tu CSV.


In [5]:
# Ruta al CSV original (ajústala según tu estructura de proyecto)
DATA_PATH = Path("../../data/youtoxic_english_1000.csv")

df = pd.read_csv(DATA_PATH)

print("Tamaño del DataFrame (filas, columnas):", df.shape)
df.head()


Tamaño del DataFrame (filas, columnas): (1000, 15)


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


## 3. Definición de columnas de interés

Definimos:

- Columnas de ID (si existen) → `CommentId`, `VideoId` (ajusta si tu dataset difiere).
- Columna de texto → `Text`.
- Columnas de etiquetas → todas las columnas que empiecen por `"Is"`.
- Columna binaria agregada `IsAnyToxic` → 1 si alguna etiqueta de toxicidad es verdadera.

Esto nos permitirá:

- Trabajar en modo **binario** (tóxico vs no tóxico).
- O escalar a clasificación **multi-etiqueta** más adelante.


In [ ]:
all_columns = df.columns.tolist()
print("Todas las columnas del dataset:")
print(all_columns)

# Ajusta estas columnas si en tu dataset tienen otros nombres
id_cols = [col for col in ["CommentId", "VideoId"] if col in df.columns]
text_col = "Text"

# Todas las columnas que empiezan por "Is" las consideramos etiquetas
label_cols = [c for c in df.columns if c.startswith("Is")]

print("\nColumnas de ID:", id_cols)
print("Columna de texto:", text_col)
print("Columnas de etiquetas:", label_cols)

# Columna binaria agregada
df["IsAnyToxic"] = df[label_cols].any(axis=1).astype(int)

df["IsAnyToxic"].value_counts(normalize=True).sort_index()


## 4. Manejo de valores nulos

Revisamos:

- Si hay textos nulos en `Text`.
- Si hay etiquetas nulas.

Acciones:

- Rellenar textos nulos con cadena vacía (`""`) para evitar errores de procesamiento.
- (Opcional) Eliminar filas completamente sin etiquetas, si existiesen.
